# Stencil F2Py Parallel

In [44]:
%%writefile stc_f2p_par.f90
subroutine stm(n, energy, niters, oheat, otime, orank)
    use MPI
    implicit none
    integer, intent(in) :: n, energy, niters
    double precision, intent(out) :: oheat, otime
    integer, intent(out) :: orank
    
    integer :: iters, i, j, px, py, rx, ry
    integer :: north, south, west, east, bx, by, offx, offy
    integer :: mpirank, mpisize, mpitag=1, mpierror
    integer, dimension(2) :: pdims=0
    integer, dimension(4) :: sendrequest, recvrequest
    double precision :: mpiwtime=0.0, heat=0.0, rheat=0.0
    double precision, dimension(:), allocatable   :: sendnorthgz, sendsouthgz
    double precision, dimension(:), allocatable   :: recvnorthgz, recvsouthgz
    double precision, dimension(:,:), allocatable :: aold, anew

    integer, parameter  :: nsources=3        ! three heat sources
    ! locnsources = number of sources in my area
    integer             :: locnsources=0, locx, locy
    ! locsources = sources local to my rank
    integer, dimension(nsources, 2) :: locsources=0, sources

    call MPI_Init(mpierror)
    call MPI_Comm_rank(MPI_COMM_WORLD, mpirank, mpierror)
    call MPI_Comm_size(MPI_COMM_WORLD, mpisize, mpierror)

    if (mpirank == 0) then
        mpiwtime = -MPI_Wtime()     ! inicializa contador de tempo
    endif
    
    ! Creates a division of processors in a Cartesian grid
    ! MPI_DIMS_CREATE(NNODES, NDIMS, DIMS, IERROR)
    !   NNODES - number of nodes in a grid
    !   NDIMS - number of Cartesian dimensions 
    !   DIMS - array specifying the number of nodes in each dimension
    ! Examples:
    !   MPI_Dims_create(6, 2, dims)  ->  (3,2)
    !   MPI_Dims_create(7, 2, dims)  ->  (7,1)
    call MPI_Dims_create(mpisize, 2, pdims, mpierror)

    ! determine my coordinates (x,y)
    px = pdims(1)
    py = pdims(2)
    rx = mod(mpirank, px)
    ry = mpirank / px

    ! determine my four neighbors
    north = (ry - 1) * px + rx; if( (ry - 1) < 0  ) north = MPI_PROC_NULL
    south = (ry + 1) * px + rx; if( (ry + 1) >= py) south = MPI_PROC_NULL
    west = ry * px + rx - 1;    if( (rx - 1) < 0  ) west  = MPI_PROC_NULL
    east = ry * px + rx + 1;    if( (rx + 1) >= px) east  = MPI_PROC_NULL

    ! decompose the domain   
    bx = n / px             ! block size in x
    by = n / py             ! block size in y
    offx = (rx * bx) + 1    ! offset in x
    offy = (ry * by) + 1    ! offset in y

    ! initialize heat sources
    sources = reshape( [ n/2,   n/2,        &
                         n/3,   n/3,        &
                         n*4/5, n*8/9 ],    &
              shape(sources), order=[2, 1])

    do i = 1, nsources      ! determine which sources are in my patch
        locx = sources(i, 1) - offx
        locy = sources(i, 2) - offy    
        if(locx >= 0 .and. locx <= bx .and. locy >= 0 .and. locy <= by) then
            locnsources = locnsources + 1
            locsources(locnsources, 1) = locx + 2
            locsources(locnsources, 2) = locy + 2
        endif
    enddo

    ! allocate communication buffers
    allocate(sendnorthgz(bx))   ! send buffers
    allocate(sendsouthgz(bx))
    allocate(recvnorthgz(bx))   ! receive buffers
    allocate(recvsouthgz(bx))
    ! allocate two work arrays
    allocate(aold(bx+2, by+2)); aold = 0.0   ! 1-wide halo zones!
    allocate(anew(bx+2, by+2)); anew = 0.0   ! 1-wide halo zones!

    ! laco principal das iteracoes
    do iters = 1, niters, 2

        ! --- anew <- stencil(aold) ---
        if(north /= MPI_PROC_NULL) then 
            sendnorthgz = aold(2, 2:bx+1)
            recvnorthgz = 0.0
            call MPI_IRecv(recvnorthgz, bx, MPI_DOUBLE_PRECISION, north,  &
                            mpitag, MPI_COMM_WORLD, recvrequest(1), mpierror)
            call MPI_ISend(sendnorthgz, bx, MPI_DOUBLE_PRECISION, north,  &
                            mpitag, MPI_COMM_WORLD, sendrequest(1), mpierror)
        endif   
        if(south /= MPI_PROC_NULL) then 
            sendsouthgz = aold(bx+1, 2:bx+1)
            recvsouthgz(:) = 0.0
            call MPI_IRecv(recvsouthgz, bx, MPI_DOUBLE_PRECISION, south,  &
                            mpitag, MPI_COMM_WORLD, recvrequest(2), mpierror)
            call MPI_ISend(sendsouthgz, bx, MPI_DOUBLE_PRECISION, south,  &
                            mpitag, MPI_COMM_WORLD, sendrequest(2), mpierror)
        endif    
        if(east /= MPI_PROC_NULL) then 
            call MPI_IRecv(aold(2:bx+1, bx+2), bx, MPI_DOUBLE_PRECISION, east, &
                            mpitag, MPI_COMM_WORLD, recvrequest(3), mpierror)
            call MPI_ISend(aold(2:bx+1, bx+1), bx, MPI_DOUBLE_PRECISION, east, &
                            mpitag, MPI_COMM_WORLD, sendrequest(3), mpierror)
        endif    
        if(west /= MPI_PROC_NULL) then 
            call MPI_IRecv(aold(2:bx+1, 1), bx, MPI_DOUBLE_PRECISION, west, &
                           mpitag, MPI_COMM_WORLD, recvrequest(4), mpierror)
            call MPI_ISend(aold(2:bx+1, 2), bx, MPI_DOUBLE_PRECISION, west, &
                           mpitag, MPI_COMM_WORLD, sendrequest(4), mpierror)
            endif
        if(north /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(1), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(1), MPI_STATUS_IGNORE, mpierror)
            aold(1, 2:bx+1)=recvnorthgz
        endif
        if(south /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(2), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(2), MPI_STATUS_IGNORE, mpierror)
            aold(bx+2, 2:bx+1)=recvsouthgz
        endif
        if(east /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(3), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(3), MPI_STATUS_IGNORE, mpierror)
        endif
        if(west /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(4), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(4), MPI_STATUS_IGNORE, mpierror)
        endif  

        ! update grid points
        do j = 2, by+1 
            do i = 2, bx+1
                anew(i, j) = aold(i, j)/2.0 + (aold(i-1, j) + aold(i+1, j) +  &
                             aold(i, j-1) + aold(i, j+1)) / 4.0 / 2.0
            enddo
        enddo

        ! adiciona calor a malha
        do i = 1, locnsources
            anew(locsources(i, 1), locsources(i, 2)) =   &
                anew(locsources(i, 1), locsources(i, 2)) + energy
        enddo

        ! --- aold <- stencil(anew) ---
        if(north /= MPI_PROC_NULL) then 
            sendnorthgz=anew(2, 2:bx+1)
            call MPI_IRecv(recvnorthgz, bx, MPI_DOUBLE_PRECISION, north, mpitag,  &
                            MPI_COMM_WORLD, recvrequest(1), mpierror)
            call MPI_ISend(sendnorthgz, bx, MPI_DOUBLE_PRECISION, north, mpitag,  &
                            MPI_COMM_WORLD, sendrequest(1), mpierror)
        endif
        if(south /= MPI_PROC_NULL) then 
            sendsouthgz=anew(bx+1, 2:bx+1)
            call MPI_IRecv(recvsouthgz, bx, MPI_DOUBLE_PRECISION, south, mpitag,  &
                            MPI_COMM_WORLD, recvrequest(2), mpierror)   
            call MPI_ISend(sendsouthgz, bx, MPI_DOUBLE_PRECISION, south, mpitag,  &
                            MPI_COMM_WORLD, sendrequest(2), mpierror)
        endif
        if(east /= MPI_PROC_NULL) then 
            call MPI_IRecv(anew(2:bx+1, bx+2), bx, MPI_DOUBLE_PRECISION, east,  &
                            mpitag, MPI_COMM_WORLD, recvrequest(3), mpierror)
            call MPI_ISend(anew(2:bx+1, bx+1), bx, MPI_DOUBLE_PRECISION, east,  &
                            mpitag, MPI_COMM_WORLD, sendrequest(3), mpierror)
        endif
        if(west /= MPI_PROC_NULL) then 
            call MPI_IRecv(anew(2:bx+1, 1), bx, MPI_DOUBLE_PRECISION, west, mpitag,  &
                            MPI_COMM_WORLD, recvrequest(4), mpierror)
            call MPI_ISend(anew(2:bx+1, 2), bx, MPI_DOUBLE_PRECISION, west, mpitag,  &
                            MPI_COMM_WORLD, sendrequest(4), mpierror)
        endif
        if(north /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(1), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(1), MPI_STATUS_IGNORE, mpierror)
            anew(1, 2:bx+1)=recvnorthgz
        endif
        if(south /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(2), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(2), MPI_STATUS_IGNORE, mpierror)
            anew(bx+2, 2:bx+1)=recvsouthgz
        endif
        if(east /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(3), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(3), MPI_STATUS_IGNORE, mpierror)
        endif
        if(west /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(4), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(4), MPI_STATUS_IGNORE, mpierror)
        endif

        ! update grid points
        do j = 2, by+1 
            do i = 2, bx+1
                aold(i, j) = anew(i, j)/2.0 + (anew(i-1, j) + anew(i+1, j) +  &
                             anew(i, j-1) + anew(i, j+1)) / 4.0 / 2.0
            enddo
        enddo

        ! adiciona calor a malha:
        do i = 1, locnsources
            aold(locsources(i, 1), locsources(i, 2)) =  &
                aold(locsources(i, 1), locsources(i, 2)) + energy
        enddo

    enddo
   
    ! ALL REDUCE:
    heat = 0.0
    do j = 2, by+1 
        do i = 2, bx+1
            heat = heat + aold(i, j)
        enddo
    enddo
    call MPI_Allreduce(heat, rheat, 1, MPI_DOUBLE_PRECISION, MPI_SUM,  &
                       MPI_COMM_WORLD, mpierror)

    orank = mpirank
    if(mpirank == 0) then
        otime = mpiwtime + MPI_Wtime()
        oheat = rheat
!        write(*, "('Heat='     f0.2' | ')", advance="no") rheat
!        write(*, "('Tempo='    f0.4' | ')", advance="no") mpiwtime
!        write(*, "('MPI_Size=' i0  ' | ')", advance="no") mpisize
!        write(*, "('MPI_Dims=('i0','i0') | ')", advance="no") pdims
!        write(*, "('bx,by=('i0','i0')')") bx,by
    
    endif

    call MPI_Finalize(mpierror)
end subroutine

Overwriting stc_f2p_par.f90


Compilado na máquina com o hardware de destino

In [45]:
%%bash
rm stc_f2p_par.*.so
module load openmpi/gnu/4.0.1
f2py -c stc_f2p_par.f90 -m stc_f2p_par --f90exec=mpif90

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "stc_f2p_par" sources
f2py options: []
f2py:> /tmp/tmp6y5mae9k/src.linux-x86_64-3.7/stc_f2p_parmodule.c
creating /tmp/tmp6y5mae9k/src.linux-x86_64-3.7
Reading fortran codes...
	Reading file 'stc_f2p_par.f90' (format:free)
Post-processing...
	Block: stc_f2p_par
			Block: stm
In: :stc_f2p_par:stc_f2p_par.f90:stm
get_useparameters: no module mpi info used by stm
Post-processing (stage 2)...
Building modules...
	Building module "stc_f2p_par"...
		Constructing wrapper function "stm"...
		  oheat,otime,orank = stm(n,energy,niters)
	Wrote C/API module "stc_f2p_par" to file "/tmp/tmp6y5mae9k/src.linux-x86_64-3.7/stc_f2p_parmodule.c"
  adding '/tmp/tmp6y5mae9k/src.linux-x86_64-3.7/fortranobject.c' to sources.
  adding '/tmp/tmp6y5mae

In file included from /scratch/app/anaconda3/2018.12/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1832:0,
                 from /scratch/app/anaconda3/2018.12/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /scratch/app/anaconda3/2018.12/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /tmp/tmp6y5mae9k/src.linux-x86_64-3.7/fortranobject.h:13,
                 from /tmp/tmp6y5mae9k/src.linux-x86_64-3.7/fortranobject.c:2:
/scratch/app/anaconda3/2018.12/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it with " \
  ^
In file included from /scratch/app/anaconda3/2018.12/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1832:0,
                 from /scratch/

In [1]:
# para funcionar precisa resetar o kernel
from stc_f2p_par import stm
print(stm.__doc__)

oheat,otime,orank = stm(n,energy,niters)

Wrapper for ``stm``.

Parameters
----------
n : input int
energy : input int
niters : input int

Returns
-------
oheat : float
otime : float
orank : int



In [2]:
%%writefile stencil_mpi.py
from time import time
from stc_f2p_par import stm

n            = 4800    # nxn grid; 4800,1,500→1500; 100,1,10→30; [4800]
energy       = 1       # energy to be injected per iteration; [1]
niters       = 500     # number of iterations; [500]
heat         = 0.0
t            = 0.0
t0           = 0.0
rank         = 0

t0 = time()
heat, t, rank = stm(n, energy, niters)
t0 = time() - t0

if not rank :
    print("Heat = %0.4f | Tempo = %0.4f | TempoPyt = %0.4f" %(heat, t, t0))

Overwriting stencil_mpi.py


In [3]:
%%bash
module load openmpi/gnu/4.0.1
mpiexec -n 4 python stencil_mpi.py

Heat = 1500.0000 | Tempo = 5.4505 | TempoPyt = 5.5963


In [4]:
%%bash
s='/prj/ampemi/eduardo.miranda2/stnc/F2Py'
d='/scratch/ampemi/eduardo.miranda2/stnc/F2Py'
rm  $d/stc_f2p_par.*.so
cp  $s/stc_f2p_par.*.so  $s/stencil_mpi.py  $d

In [18]:
%%writefile stc_f2p_par_81.srm
#!/bin/bash
# limites das filas (1,0 UA):
#   cpu_dev  : 20 min.,  1-4  nós, 1/1   tarefas em exec/fila máximo
#   cpu_small: 72 horas, 1-20 nós, 16/96 tarefas em exec/fila máximo
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81

#SBATCH --ntasks=81            #Total de tarefas
#SBATCH -p cpu_small           #Fila (partition) a ser utilizada
#SBATCH -J stf2pa              #Nome do job, 8 caracteres
#SBATCH --time=00:02:00        #Tempo max. de execução 2 minutos
# #SBATCH --nodes=5              #Qtd de nós
# #SBATCH --ntasks-per-node=5    #Qtd de tarefas por nó ($SLURM_NTASKS_PER_NODE)
# #SBATCH --exclusive            #Utilização exclusiva dos nós

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tarefas por no:' $SLURM_NTASKS_PER_NODE
echo '- Qtd. de nos:' $SLURM_JOB_NUM_NODES
echo '- Tot. de tarefas:' $SLURM_NTASKS
echo '- Nos alocados:' $SLURM_JOB_NODELIST
echo '- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):'
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR
nodeset -e $SLURM_JOB_NODELIST

#Configura o ambiente
echo '-- modulos ----------------------------'
echo 'module load openmpi/gnu/4.0.1'
module load openmpi/gnu/4.0.1

#Acessa o diretório onde o script está localizado 
cd /scratch/ampemi/eduardo.miranda2/stnc/F2Py/

#Configura o executavel
EXEC='python stencil_mpi.py'

#Dispara a execucao
echo '-- srun -------------------------------'
echo '$ srun --mpi=pmi2 -n' $SLURM_NTASKS $EXEC
srun --mpi=pmi2 -n $SLURM_NTASKS $EXEC
echo '-- FIM --------------------------------'

Overwriting stc_f2p_par_81.srm


### Testa se funciona

In [6]:
%%bash
sbatch stc_f2p_par_04.srm

Submitted batch job 781260


In [7]:
! squeue -n stf2pa  # verifica se já terminou

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            781260 cpu_small   stf2pa eduardo. PD       0:00      1 (Priority)


In [9]:
! squeue -n stf2pa  # verifica se já terminou

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [10]:
%%bash
d='/scratch/ampemi/eduardo.miranda2/stnc/F2Py'
cat $d/slurm-781260.out

- Job ID: 781260
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 4
- Nos alocados: sdumont1124
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/F2Py
sdumont1124
-- modulos ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 4 python stencil_mpi.py
Heat = 1500.0000 | Tempo = 7.4491 | TempoPyt = 7.6862
-- FIM --------------------------------


### Envia demais

In [19]:
%%bash
# 1x1=1, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
sbatch stc_f2p_par_01.srm
sbatch stc_f2p_par_09.srm
sbatch stc_f2p_par_16.srm
sbatch stc_f2p_par_25.srm  # enviado por engano
sbatch stc_f2p_par_36.srm
sbatch stc_f2p_par_49.srm
sbatch stc_f2p_par_64.srm
sbatch stc_f2p_par_81.srm

Submitted batch job 781265
Submitted batch job 781266
Submitted batch job 781267
Submitted batch job 781268
Submitted batch job 781269
Submitted batch job 781270
Submitted batch job 781271
Submitted batch job 781272


In [20]:
! squeue -n stf2pa  # verifica se já terminou

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            781265 cpu_small   stf2pa eduardo.  R       0:05      1 sdumont1186
            781266 cpu_small   stf2pa eduardo.  R       0:05      1 sdumont1328
            781267 cpu_small   stf2pa eduardo.  R       0:05      1 sdumont1329
            781269 cpu_small   stf2pa eduardo.  R       0:05      2 sdumont[1206-1207]
            781270 cpu_small   stf2pa eduardo.  R       0:05      3 sdumont[1208-1210]
            781271 cpu_small   stf2pa eduardo.  R       0:05      3 sdumont[1211-1213]
            781272 cpu_small   stf2pa eduardo.  R       0:05      4 sdumont[1214-1217]


In [21]:
! squeue -n stf2pa  # verifica se já terminou

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [22]:
%%bash
d='/scratch/ampemi/eduardo.miranda2/stnc/F2Py'
cat $d/slurm-781265.out  #1
cat $d/slurm-781260.out  #4
cat $d/slurm-781266.out  #9
cat $d/slurm-781267.out  #16
cat $d/slurm-781269.out  #36
cat $d/slurm-781270.out  #49
cat $d/slurm-781271.out  #64
cat $d/slurm-781272.out  #81

- Job ID: 781265
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1186
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/F2Py
sdumont1186
-- modulos ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 python stencil_mpi.py
Heat = 1500.0000 | Tempo = 23.4926 | TempoPyt = 23.6593
-- FIM --------------------------------
- Job ID: 781260
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 4
- Nos alocados: sdumont1124
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/F2Py
sdumont1124
-- modulos ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 4 python stencil_mpi.py
Heat = 1500.0000 | Tempo = 7.4491 | TempoPyt = 7.6862
-- FIM --------------------------------
- Job ID: 781266
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 9
- Nos alo

# Segunda tomada de tempo

In [2]:
%%bash
# 1x1=1, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
sbatch stc_f2p_par_01.srm
sbatch stc_f2p_par_04.srm
sbatch stc_f2p_par_09.srm
sbatch stc_f2p_par_16.srm
sbatch stc_f2p_par_36.srm
sbatch stc_f2p_par_49.srm
sbatch stc_f2p_par_64.srm
sbatch stc_f2p_par_81.srm

Submitted batch job 788047
Submitted batch job 788048
Submitted batch job 788049
Submitted batch job 788050
Submitted batch job 788051
Submitted batch job 788052
Submitted batch job 788053
Submitted batch job 788054


In [3]:
! squeue -n stf2pa  # verifica se já terminou

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788047 cpu_small   stf2pa eduardo. PD       0:00      1 (Priority)
            788048 cpu_small   stf2pa eduardo. PD       0:00      1 (Priority)
            788049 cpu_small   stf2pa eduardo. PD       0:00      1 (Priority)
            788050 cpu_small   stf2pa eduardo. PD       0:00      1 (Priority)
            788051 cpu_small   stf2pa eduardo. PD       0:00      2 (Priority)
            788052 cpu_small   stf2pa eduardo. PD       0:00      3 (Priority)
            788053 cpu_small   stf2pa eduardo. PD       0:00      3 (Priority)
            788054 cpu_small   stf2pa eduardo. PD       0:00      4 (Priority)


In [3]:
! squeue -n stf2pa  # verifica se já terminou

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [4]:
%%bash
d='/scratch/ampemi/eduardo.miranda2/stnc/F2Py'
cat $d/slurm-788047.out  #1
cat $d/slurm-788048.out  #4
cat $d/slurm-788049.out  #9
cat $d/slurm-788050.out  #16
cat $d/slurm-788051.out  #36
cat $d/slurm-788052.out  #49
cat $d/slurm-788053.out  #64
cat $d/slurm-788054.out  #81

- Job ID: 788047
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/F2Py
sdumont1149
-- modulos ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 python stencil_mpi.py
Heat = 1500.0000 | Tempo = 23.6711 | TempoPyt = 23.8236
-- FIM --------------------------------
- Job ID: 788048
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 4
- Nos alocados: sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/F2Py
sdumont1149
-- modulos ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 4 python stencil_mpi.py
Heat = 1500.0000 | Tempo = 7.4518 | TempoPyt = 7.6480
-- FIM --------------------------------
- Job ID: 788049
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 9
- Nos alo

# Terceira tomada de tempo

In [5]:
%%bash
# 1x1=1, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
sbatch stc_f2p_par_01.srm
sbatch stc_f2p_par_04.srm
sbatch stc_f2p_par_09.srm
sbatch stc_f2p_par_16.srm
sbatch stc_f2p_par_36.srm
sbatch stc_f2p_par_49.srm
sbatch stc_f2p_par_64.srm
sbatch stc_f2p_par_81.srm

Submitted batch job 788055
Submitted batch job 788056
Submitted batch job 788057
Submitted batch job 788058
Submitted batch job 788059
Submitted batch job 788060
Submitted batch job 788061
Submitted batch job 788062


In [6]:
! squeue -n stf2pa  # verifica se já terminou

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788053 cpu_small   stf2pa eduardo. PD       0:00      3 (Resources)
            788054 cpu_small   stf2pa eduardo. PD       0:00      4 (Resources)
            788051 cpu_small   stf2pa eduardo. PD       0:00      2 (Resources)
            788052 cpu_small   stf2pa eduardo. PD       0:00      3 (Resources)
            788050 cpu_small   stf2pa eduardo. PD       0:00      1 (Resources)
            788049 cpu_small   stf2pa eduardo. PD       0:00      1 (Resources)
            788048 cpu_small   stf2pa eduardo. PD       0:00      1 (Resources)
            788047 cpu_small   stf2pa eduardo. PD       0:00      1 (Resources)
            788055 cpu_small   stf2pa eduardo. PD       0:00      1 (Priority)
            788056 cpu_small   stf2pa eduardo. PD       0:00      1 (Priority)
            788057 cpu_small   stf2pa eduardo. PD       0:00      1 (Priority)
            788058 cpu_small   stf2pa 

In [1]:
! squeue -n stf2pa  # verifica se já terminou

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [2]:
%%bash
d='/scratch/ampemi/eduardo.miranda2/stnc/F2Py'
cat $d/slurm-788055.out  #1
cat $d/slurm-788056.out  #4
cat $d/slurm-788057.out  #9
cat $d/slurm-788058.out  #16
cat $d/slurm-788059.out  #36
cat $d/slurm-788060.out  #49
cat $d/slurm-788061.out  #64
cat $d/slurm-788062.out  #81

- Job ID: 788055
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/F2Py
sdumont1149
-- modulos ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 python stencil_mpi.py
Heat = 1500.0000 | Tempo = 23.6284 | TempoPyt = 23.7599
-- FIM --------------------------------
- Job ID: 788056
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 4
- Nos alocados: sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/F2Py
sdumont1149
-- modulos ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 4 python stencil_mpi.py
Heat = 1500.0000 | Tempo = 7.4559 | TempoPyt = 7.6475
-- FIM --------------------------------
- Job ID: 788057
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 9
- Nos alo